## Init e dependencies

In [26]:
! pip install langchain==0.2.7 langchain_aws langchain-community langchain_core boto3 botocore
! pip install PyMuPDF python-docx pydantic

### Init LLM endpoints




In [33]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


### Caricamento Files

In [34]:
import fitz  # PyMuPDF
import docx

from google.colab import files

def upload_file():
  uploaded = files.upload()
  # Una volta eseguita questa cella, Colab ti chiederà di selezionare il file dal tuo PC.

  # 2) Identifica il nome del file caricato
  for file_name in uploaded.keys():
      file_path = file_name

  return file_path

def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text

def read_txt(file_path):
    with open(file_path, "r") as f:
        text = f.read()
    return text

def upload_and_read_pdf():
    file_path = upload_file()

    return read_pdf(file_path)

def upload_and_read_docx():
    file_path = upload_file()

    return read_docx(file_path)

def upload_and_read_txt():
    file_path = upload_file()

    return read_txt(file_path)

In [ ]:
Summarization

In [37]:
def summarize(text):

    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="""
            Riassumi il seguente testo:
            {text}

            REGOLE
            - scrivi una breve intro per spiegare il contesto
            - riassumi i punti principali in una bullet list
            - trai le tue conclusioni sull'argomento
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"text": text})
    return response


### Esempio - 1

In [36]:

text = upload_and_read_pdf()

response = summarize(text)

Saving RomeCall_Paper_web.pdf to RomeCall_Paper_web (3).pdf


In [38]:
from IPython.display import display, Markdown
display(Markdown(f"### Riassunto:\n\n{response}"))


### Riassunto:

Introduzione
Il testo presenta la "Rome Call for AI Ethics", un documento firmato nel 2020 per promuovere un approccio etico all'intelligenza artificiale. L'obiettivo è di creare un futuro in cui l'innovazione digitale e il progresso tecnologico mettano l'essere umano al centro.

Punti principali:
- L'IA offre grandi potenzialità per migliorare il benessere sociale e personale, ma deve essere sviluppata nel rispetto della dignità umana e senza avere come unico scopo il profitto.
- I firmatari si impegnano a promuovere uno sviluppo dell'IA al servizio dell'intera umanità, che non discrimini nessuno e che sia attento all'ambiente.
- Vengono definiti 6 principi etici fondamentali per l'IA: trasparenza, inclusione, responsabilità, imparzialità, affidabilità, sicurezza e privacy.
- Viene sottolineata l'importanza dell'istruzione e della formazione per sensibilizzare sulle opportunità e sui rischi dell'IA.
- Viene evidenziata la necessità di sviluppare nuove forme di regolamentazione per promuovere la trasparenza e il rispetto dei principi etici.

Conclusioni
Il documento rappresenta un importante passo avanti nella definizione di un quadro etico per lo sviluppo dell'intelligenza artificiale, ponendo l'essere umano e l'ambiente al centro. L'impegno congiunto di organizzazioni internazionali, governi e aziende tecnologiche è fondamentale per realizzare questa visione e garantire che l'innovazione digitale serva il bene dell'umanità.

Esempio - 2

In [40]:
text = upload_and_read_txt()

summary = summarize(text)

display(Markdown(f"### Riassunto:\n\n{response}"))

Saving commedia.txt to commedia (2).txt


ValueError: Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: Input is too long for requested model.

In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50000, chunk_overlap=500)
chunks = text_splitter.split_text(text)

print(len(chunks))


12


In [51]:
def summarize_more(text):

    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="""
            Riassumi il seguente testo:
            {text}

            REGOLE
            - per ogni canto ritorna max due righe
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"text": text})
    return response

summaries = []
for chunk in chunks:
    response = summarize_more(chunk)
    summaries.append(response)


In [52]:
summary = "\n".join(summaries)
display(Markdown(f"### Riassunto:\n\n{summary}"))

### Riassunto:

Ecco un breve riassunto dei principali eventi dei Canti I-X dell'Inferno di Dante:

Canto I: Il poeta si ritrova in una selva oscura e smarrita. Incontra Virgilio, che gli si offre come guida per l'Inferno.

Canto II: Beatrice manda Virgilio a soccorrere Dante, che esita ad affrontare il viaggio. Virgilio lo convince a seguirlo.

Canto III: Dante e Virgilio entrano nell'Inferno, leggendo l'iscrizione sulla porta che annuncia la disperazione eterna. Incontrano le anime dei pusillanimi.

Canto IV: Dante e Virgilio giungono al Limbo, dove trovano i grandi spiriti dell'antichità, tra cui Omero, Orazio, Ovidio e Lucano.

Canto V: Incontrano le anime dei lussuriosi, tra cui Francesca da Rimini, che racconta la sua tragica storia d'amore.

Canto VI: Giungono nel terzo cerchio, dove sono puniti i golosi, tormentati dalla pioggia, dal freddo e da Cerbero.

Canto VII: Nel quarto cerchio, Dante vede i prodighi e gli avari rotolare pesi. Virgilio spiega il ruolo della Fortuna.

Canto VIII: Attraversano la palude Stigia, dove incontrano Filippo Argenti. Giungono alle mura di Dite, ma i demoni ne impediscono l'ingresso.

Canto IX: Virgilio torna solo e spiega a Dante che dovrà affrontare da solo l'ingresso di Dite. Appaiono le Furie.

Canto X: Entrano nella città di Dite, dove sono puniti gli eretici, rinchiusi in sepolcri infuocati.
Ecco un breve riassunto dei principali eventi dei Canti X-XIX dell'Inferno di Dante:

Canto X: Dante e Virgilio incontrano Farinata degli Uberti, un eretico condannato all'Inferno. Farinata profetizza l'esilio di Dante da Firenze.

Canto XI: Virgilio spiega a Dante la struttura dell'Inferno, dividendolo in cerchi in base alla gravità dei peccati.

Canto XII: Dante e Virgilio scendono nel cerchio dei violenti, dove trovano i centauri che sorvegliano un fiume di sangue bollente.

Canto XIII: Nel bosco dei suicidi, Dante parla con un'anima che si è trasformata in un albero.

Canto XIV: Dante e Virgilio incontrano Capaneo, un bestemmiatore, e scorgono il Minotauro, guardiano del cerchio dei violenti.

Canto XV: Dante incontra il suo maestro Brunetto Latino, condannato per sodomia, e apprende il suo destino a Firenze.

Canto XVI: Dante parla con tre famosi fiorentini, Guido Guerra, Tegghiaio Aldobrandi e Iacopo Rusticucci, condannati per violenza.

Canto XVII: Dante e Virgilio cavalcano il mostro Gerione per scendere nel cerchio degli usurai.

Canto XVIII: Giunti a Malebolge, Dante vede i ruffiani frustati dai demoni.

Canto XIX: Dante e Virgilio scendono nel cerchio dei simoniaci, dove Dante riconosce un suo conoscente.
Ecco un breve riassunto dei canti dell'Inferno:

Canto XIX: Dante e Virgilio incontrano i simoniaci, puniti nella terza bolgia. Qui Dante parla con Papa Bonifacio VIII, che lo scambia per il suo successore Clemente V.

Canto XX: Nella quarta bolgia, Dante vede gli indovini, le cui teste sono rivolte all'indietro. Virgilio spiega la storia di Manto e la fondazione di Mantova.

Canto XXI: Dante e Virgilio giungono all'argine della quinta bolgia, dove sono puniti i barattieri. I diavoli Malebranche li accolgono con minacce, ma Virgilio riesce a farli passare.

Canto XXII: Nella quinta bolgia, Dante vede i barattieri immersi in una pece bollente. Assiste alla trasformazione di un barattiere in serpente e viceversa.

Canto XXIII: Dante e Virgilio proseguono il loro cammino. Nella sesta bolgia, Dante vede i falsari, trasformati in orribili creature.

Canto XXIV: Dante e Virgilio salgono sulla settima bolgia, dove sono puniti i ladri, trasformati in serpenti.

Canto XXV: Dante assiste a ulteriori metamorfosi di dannati, tra cui Vanni Fucci, che profetizza sventure per Firenze.

Canto XXVI: Dante e Virgilio giungono all'ottava bolgia, dove sono puniti i consiglieri fraudolenti. Qui Dante incontra Ulisse e Diomede.

Canto XXVII: Nella stessa bolgia, Dante parla con un altro dannato, Guido da Montefeltro, che gli racconta la sua storia.
Ecco un breve riassunto del Canto III del Purgatorio:

Dante e Virgilio si ritrovano sulla spiaggia del Purgatorio, dove incontrano un gruppo di anime che stanno aspettando di poter iniziare la loro ascesa. Tra loro c'è Casella, un musicista amico di Dante, che canta una canzone d'amore. Tutti si fermano ad ascoltarlo, finché un vecchio li richiama all'ordine, esortandoli a salire sul monte per iniziare la loro purificazione. A quel punto le anime si affrettano a raggiungere la costa, abbandonando il canto.
Ecco un breve riassunto dei principali eventi dei Canti III-XII del Purgatorio:

Canto III: Dante e Virgilio incontrano le anime dei morti che si sono pentiti in punto di morte. Tra loro c'è Manfredi, nipote dell'imperatrice Costanza, che chiede a Dante di informare sua figlia della sua sorte.

Canto IV: Dante e Virgilio salgono il monte del Purgatorio, incontrando anime che attendono di poter salire. Tra loro c'è Belacqua, che spiega di dover attendere un certo periodo di tempo prima di poter iniziare la sua purificazione.

Canto V: Dante e Virgilio incontrano un gruppo di anime che pregano per essere liberate dai loro peccati. Tra loro c'è Pia de' Tolomei, che chiede a Dante di ricordare il suo nome alla sua famiglia.

Canto VI: Dante critica aspramente l'Italia, divisa e in preda a lotte intestine, e Firenze, sua città natale, per la sua instabilità politica.

Canto VII: Dante e Virgilio incontrano anime illustri, come Rodolfo d'Asburgo e Ottocaro di Boemia, che scontano le loro colpe nel Purgatorio.

Canto VIII: Dante e Virgilio assistono all'arrivo di due angeli che cacciano un serpente, simbolo del male, dalla valletta in cui si trovano le anime.

Canto IX: Dante e Virgilio giungono alla porta del Purgatorio, dove un angelo li accoglie e li fa entrare dopo averli purificati.

Canto X: Dante e Virgilio ammirano i bassorilievi che decorano la parete del Purgatorio, raffiguranti episodi di umiltà e superbia.

Canto XI: Le anime del Purgatorio pregano per ottenere la remissione dei loro peccati. Dante incontra Omberto Aldobrandeschi e Oderisi da Gubbio, artisti del passato.

Canto XII: Dante e Virgilio proseguono la salita, incontrando altre anime che scontano la pena della superbia.
Ecco un breve riassunto dei Canti XII-XXI del Purgatorio:

Canto XII: Dante e Virgilio salgono il monte del Purgatorio, ammirando le raffigurazioni scolpite sulla parete. Vengono raggiunti da un angelo che li invita a salire più in alto.

Canto XIII: Dante e Virgilio giungono al secondo girone, dove sono puniti gli invidiosi. Sentono voci che cantano esempi di carità e di invidia punita.

Canto XIV: Dante incontra l'anima di Guido del Duca, che deplora la decadenza morale della Romagna. Poi sente una voce che annuncia l'arrivo di un angelo.

Canto XV-XVI: Dante e Virgilio proseguono il cammino, incontrando anime che piangono il loro peccato di avarizia. Il poeta apprende da Virgilio la natura dell'amore e del libero arbitrio.

Canto XVII-XVIII: Dante ha una visione allegorica della Sirena, simbolo della seduzione mondana, che viene svelata dalla Donna santa. Poi sente voci che incitano alla fretta di raggiungere la salvezza.

Canto XIX-XX: Dante incontra l'anima di Papa Adriano V, che spiega come l'avarizia sia punita in quel girone. Poi il monte trema e si ode un inno di gloria.

Canto XXI: Dante e Virgilio proseguono il cammino, ascoltando le anime che piangono i loro peccati di avarizia.
Ecco un breve riassunto del testo:

Nel Purgatorio, Dante e Virgilio incontrano un'ombra che si rivela essere Stazio. Stazio racconta la sua storia e come la lettura dell'Eneide di Virgilio lo abbia portato alla fede cristiana. Poi Virgilio spiega a Dante il processo di formazione dell'anima umana. Infine, Stazio e Virgilio guidano Dante attraverso il fuoco purificatore.

Dante incontra anche l'anima di Guido Guinizzelli, che lo elogia come miglior poeta d'amore. Poco dopo, Dante ha una visione di Lia e Rachele, simboli della vita attiva e contemplativa. Giunti in cima al Purgatorio, Virgilio lascia Dante, affidandolo alle cure di Beatrice.
Ecco un breve riassunto dei principali eventi dei Canti III, IV e V del Paradiso:

Canto III: Dante incontra l'anima di Piccarda Donati, che gli spiega come le anime beate siano pienamente appagate dalla volontà di Dio, senza desiderare nulla di più. Piccarda racconta la sua storia, come fu costretta a lasciare il convento.

Canto IV: Beatrice risponde ai dubbi di Dante sulla giustizia divina, spiegando come la volontà umana, anche se violentata, non perde il suo valore agli occhi di Dio. Discute anche della teoria platonica sulla reincarnazione delle anime.

Canto V: Beatrice continua a spiegare il valore del voto e del libero arbitrio. Spiega che il voto è un sacrificio prezioso agli occhi di Dio, ma che può essere sostituito con il permesso della Chiesa. Ammonisce i cristiani a non essere leggeri nei loro voti.
Ecco un breve riassunto dei principali eventi dei Canti VI-XV del Paradiso:

Canto VI: Giustiniano racconta la storia dell'aquila imperiale romana, dalla sua fondazione fino ai suoi grandi successi. Critica i Ghibellini e i Guelfi per il loro uso improprio del simbolo dell'aquila.

Canto VII: Beatrice spiega come la redenzione dell'uomo attraverso l'incarnazione di Cristo sia stata la soluzione più degna al peccato originale.

Canto VIII: Dante incontra l'anima di Carlo Martello, che gli spiega come la diversità degli uomini sia voluta da Dio per il bene dell'umanità.

Canto IX: Cunizza da Romano e Folco di Marsiglia lodano l'operato di San Domenico, fondatore dell'Ordine dei Predicatori.

Canto X: San Tommaso d'Aquino spiega come la perfezione umana si realizzi nell'unione della natura divina e umana in Cristo.

Canto XI-XII: San Bonaventura elogia San Francesco d'Assisi e il suo Ordine, contrapponendolo alla decadenza di alcuni seguaci.

Canto XIII-XIV: Dante contempla la Trinità e la gloria dei beati, ammirando la bellezza della croce di luce che risplende in Paradiso.

Canto XV: Dante incontra il suo antenato Cacciaguida, che gli profetizza l'esilio e lo esorta ad affrontarlo con coraggio.
Ecco un breve riassunto dei principali eventi dei Canti XV-XXIV del Paradiso di Dante:

Canto XV: Dante incontra il suo antenato Cacciaguida, che gli rivela la storia della sua famiglia e della Firenze del passato, prima che fosse corrotta.

Canto XVI: Cacciaguida profetizza a Dante l'esilio e le difficoltà che dovrà affrontare, ma lo incoraggia a non temere.

Canto XVII: Cacciaguida spiega a Dante come la Provvidenza divina conosca il futuro, pur senza determinarlo. Preannuncia l'imminente esilio di Dante.

Canto XVIII: Dante vede le anime dei contemplativi, tra cui San Pier Damiani, che critica aspramente la corruzione del clero.

Canto XIX: Pier Damiani spiega come la salvezza dipenda dalla fede in Cristo, non dalle opere. Condanna la cupidigia del clero.

Canto XX: Dante vede le anime di altri contemplativi, tra cui Benedetto e Macario. Ammira la bellezza del Paradiso.

Canto XXI: Un'anima beata, identificata come San Pier Damiani, spiega a Dante il motivo per cui non si canta nel cielo di Saturno.

Canto XXII: Dante sale al cielo delle Stelle Fisse. Vede le anime dei contemplativi e San Benedetto, che critica la decadenza dei monasteri.

Canto XXIII: Dante contempla la gloriosa processione di Santi e Angeli, guidati dalla Vergine Maria.

Canto XXIV: Inizia l'esame sulla fede, condotto dall'apostolo San Pietro.
Ecco un breve riassunto dei principali contenuti dei Canti XXIV-XXXIII del Paradiso:

Canto XXIV: Il poeta è interrogato sulla fede da San Pietro, che lo esamina sulla sua credenza. Dante risponde in modo soddisfacente, dimostrando la sua profonda conoscenza della dottrina cristiana.

Canto XXV: Dante è incoronato poeta e riceve l'investitura da San Pietro. Appare poi l'anima di San Giacomo, che lo interroga sulla speranza.

Canto XXVI: Appare l'anima di Adamo, che spiega a Dante l'origine del male e la creazione degli angeli.

Canto XXVII: Dante assiste a una dura invettiva di San Pietro contro la corruzione della Chiesa.

Canto XXVIII: Dante contempla la struttura gerarchica del Paradiso, con i beati disposti in una rosa mistica.

Canto XXIX: San Bernardo spiega a Dante la disposizione dei beati nella rosa celeste.

Canto XXX: Dante ammira la bellezza della rosa mistica e la luce divina che la irradia.

Canto XXXI-XXXIII: San Bernardo guida Dante alla visione finale di Dio, culminante nell'ultima preghiera alla Vergine Maria.
Ecco un breve riassunto del Canto XXXIII del Paradiso di Dante:

La Vergine Maria è lodata come nobile e alta creatura, che ha nobilitato la natura umana. In lei si è acceso l'amore divino, ed è fonte di carità e speranza per i mortali. Il poeta chiede alla Vergine di purificarlo dalla sua mortalità, affinché possa elevarsi alla visione suprema. Beatrice e i beati intercedono per lui. Il poeta raggiunge la visione di Dio, una luce infinita e incomprensibile, in cui vede riflessa l'immagine dell'umanità. Infine, il poeta contempla il mistero della Trinità, espresso in tre cerchi di luce.